<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork820-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Support Vector Machine**


Tempo estimado necessário: **30** minutos


Neste laboratório, você aprenderá e obterá práticas práticas no modelo Support Vector Machine.


Usaremos um conjunto de dados de sugestão de itens alimentares para diabetes do mundo real, que contém informações nutricionais detalhadas sobre um item alimentar. O objetivo é classificar qual alimento um paciente diabético deve escolher com mais ou menos frequência para um item alimentar específico, considerando seus nutrientes.


## Objetivos


Depois de concluir este laboratório, você será capaz de:


* Treinar e avaliar classificadores SVM
* Ajuste hiperparâmetros SVM importantes, como regularização e tipos de kernel
* Plotar hiperplanos e margens de modelos SVM treinados



## SVM Overview


O SVM tenta encontrar hiperplanos que tenham a margem máxima. Os hiperplanos são determinados por vetores de suporte (os pontos de dados têm a menor distância dos hiperplanos). Enquanto isso, a fim de reduzir a variância do modelo, o modelo SVM visa encontrar o máximo de margens possíveis para que os dados não vistos tenham maior probabilidade de serem classificados corretamente.



<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/module3_svm/images/svm.png" width="720" alt="evaluation metrics">
</center>


SVM aborda não-linear separável via truque de kernel. Kernels são um tipo especial de função que pega dois vetores e retorna um número real, como uma operação dot-product. Como tal, os núcleos não são quaisquer funções de mapeamento reais de espaços de baixa dimensão para espaços de alta dimensão.


***


## Ambiente de laboratório de configuração


In [ ]:
# Import required packages
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
# Evaluation metrics related methods
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_recall_fscore_support, precision_score, recall_score

import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
%matplotlib inline

In [ ]:
# Setup a random seed to be 123
rs = 123

In [ ]:
# Load the dataset
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/datasets/food_items_binary.csv"
food_df = pd.read_csv(dataset_url)

and let's quickly looks at its first 5 rows


In [ ]:
food_df.head(10)

In [ ]:
# Get the row entries with col 0 to -1 (16)
feature_cols = list(food_df.iloc[:, :-1].columns)
feature_cols

In [ ]:
X = food_df.iloc[:, :-1]
y = food_df.iloc[:, -1:]

In [ ]:
X.describe()

In [ ]:
# # Get the row entries with the last col 'class'
y.value_counts(normalize=True)

In [ ]:
y.value_counts().plot.bar(color=['red', 'green'])

## Criar um modelo SVM com parâmetros padrão


Primeiro, vamos dividir o conjunto de dados de treinamento e teste. O conjunto de dados de treinamento será usado para treinar e ajustar modelos, e o conjunto de dados de teste será usado para avaliar os modelos. Observe que você também pode dividir um conjunto de dados de validação do conjunto de dados de treinamento somente para ajuste de modelo.


In [ ]:
# First, let's split the training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = rs)

In [ ]:
model = SVC()

In [ ]:
model.fit(X_train, y_train.values.ravel())

In [ ]:
preds = model.predict(X_test)

In [ ]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp, average='binary')
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

In [ ]:
evaluate_metrics(y_test, preds)

## Treine SVM com diferentes parâmetros de regularização e kernels


In [ ]:
model = SVC(C=10, kernel='rbf')
model.fit(X_train, y_train.values.ravel())
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)

In [ ]:
params_grid = {
    'C': [1, 10, 100],
    'kernel': ['poly', 'rbf', 'sigmoid']
}

In [ ]:
model = SVC()

In [ ]:
# Define a GridSearchCV to search the best parameters
grid_search = GridSearchCV(estimator = model, 
                           param_grid = params_grid, 
                           scoring='f1',
                           cv = 5, verbose = 1)
# Search the best parameters with training data
grid_search.fit(X_train, y_train.values.ravel())
best_params = grid_search.best_params_

In [ ]:
best_params

In [ ]:
model = SVC(C=100, kernel='rbf')
model.fit(X_train, y_train.values.ravel())
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)

The best f1score now becomes `0.88` after hyperparameter tuning.


## Plotar hiperplano SVM e margem


In [ ]:
simplified_food_df = food_df[['Calories', 'Dietary Fiber', 'class']]

In [ ]:
X = simplified_food_df.iloc[:1000, :-1].values
y = simplified_food_df.iloc[:1000, -1:].values

In [ ]:
under_sampler = RandomUnderSampler(random_state=123)
X_under, y_under = under_sampler.fit_resample(X, y)

In [ ]:
print(f"Dataset resampled shape, X: {X_under.shape}, y: {y_under.shape}")

In [ ]:
linear_svm = SVC(C=1000, kernel='linear')
linear_svm.fit(X_under, y_under)

In [ ]:
def plot_decision_boundry(X, y, model):
    plt.figure(figsize=(16, 12))
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired)

    # plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = model.decision_function(xy).reshape(XX.shape)

    # plot decision boundary and margins
    ax.contour(
        XX, YY, Z, colors="k", levels=[-1, 0, 1], alpha=0.5, linestyles=["--", "-", "--"]
    )

    # plot support vectors
    ax.scatter(
        model.support_vectors_[:, 0],
        model.support_vectors_[:, 1],
        s=100,
        linewidth=1,
        facecolors="none",
        edgecolors="k",
    )
    plt.show()

In [ ]:
plot_decision_boundry(X_under, y_under, linear_svm)

In [ ]:
svm_rbf_kernel = SVC(C=100, kernel='rbf')
svm_rbf_kernel.fit(X_under, y_under)

In [ ]:
plot_decision_boundry(X_under, y_under, svm_rbf_kernel)

## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-9-23         | 1.0     | Yan        | Created the initial version |
| 2022-2-9          | 1.1     | Steve Hord | QA pass                     |
